In [1]:
import numpy as np
import cv2
import torch
import math
import matplotlib.pyplot as plt
import warnings
import gradio as gr

In [2]:
from matplotlib import pyplot as plt
from argparser import parse_arguments
from PIL import Image
from main import get_transforms
from dataloader import imfol
from dataloader.imfol import ImageFolder, make_dataset
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data import DataLoader
from torch.autograd import Variable
from utils.visualize import vis_patch, vis_image
from models import texturegan,discriminator
from train import gen_input, rand_between, gen_input_rand
from ipywidgets import interact, interactive, fixed, interact_manual
from utils import transforms as custom_transforms

In [3]:
!pip install gradio

In [4]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

In [5]:
command = '--display_port 7770 --load 0 --load_D -1 --load_epoch 105 --gpu 2 --model texturegan --feature_weight 1e2 --pixel_weight_ab 1e3 --global_pixel_weight_l 1e3 --local_pixel_weight_l 0 --style_weight 0 --discriminator_weight 1e3 --discriminator_local_weight 1e6  --learning_rate 1e-4 --learning_rate_D 1e-4 --batch_size 36 --save_every 50 --num_epoch 100000 --save_dir /hdd/2017IS013/researchGroup03/Pytorch_ARGAN/data/train_txt --load_dir /hdd/2017IS013/researchGroup03/Pytorch_ARGAN/data/train_txt --data_path /hdd/2017IS013/researchGroup03/Pytorch_ARGAN/data/ --learning_rate_D_local  1e-4 --local_texture_size 50 --patch_size_min 20 --patch_size_max 40 --num_input_texture_patch 1 --visualize_every 5 --num_local_texture_patch 1'
args = parse_arguments(command.split())
    
args.batch_size = 1
args.image_size =152
args.resize_max = 256
args.resize_min = 256
args.data_path = '/hdd/2017IS013/researchGroup03/Pytorch_ARGAN/data/' #change to your data path

In [6]:
transform = get_transforms(args)

In [7]:
#defaulty used

def convertToStandard(input_img, texture):
#convert input sketch to canny 
#path should be changed here
#     input_img = cv2.imread("data/78.jpeg") #path to cloth folder
    #plt.imshow(input_img)
    
    
    
# args.data_path = '/hdd/2017IS013/researchGroup03/Pytorch_ARGAN/data/' #change to your data path
    #img_path = "data/blank.jpg"
    img_path = "C:/Users/ASUS/Desktop/OurGAN/data/blank.jpg"
    plt.imshow(img_path)
    #path should be changed here
    #input_img = cv2.imread("C:/Users/ASUS/Desktop/OurGAN/data/78.jpeg") #path to cloth folder
    
    gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray,100,200)
    ret,th2 = cv2.threshold(edges,100,255,cv2.THRESH_BINARY_INV)
    sketch_image = "C:/Users/ASUS/Desktop/OurGAN/data/sketchmyfrock6.jpg"  #path to save sketches images
    #sketch_image = "data/sketchmyfrock6.jpg"  #path to save sketches images
    cv2.imwrite(sketch_image, th2)
    
    skg_path = sketch_image
    img = cv2.imread(skg_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #plt.imshow(gray)
    thresh = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
    #plt.imshow(thresh)
    edges = cv2.dilate(cv2.Canny(thresh,0,255), None)
    #plt.imshow(edges)
    cnt = sorted(cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]
    
    masked = []
    segmented = []

    mask = np.zeros((256,256), np.uint8)
    masked.append(cv2.drawContours(mask, [cnt], -1, 255, -1))

    dst = cv2.bitwise_and(img, img, mask=mask)
    segmented = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    filename = "C:/Users/ASUS/Desktop/OurGAN/data/seg.jpg"
    #filename = "data/seg.jpg"
    cv2.imwrite(filename, mask)
    
    seg_path = filename
    eroded_seg_path = filename
    #Set the texture patch path here
    txt_path = "C:/Users/ASUS/Desktop/OurGAN/data/sea.jpeg"
    #txt_path = "data/texture.jpg"
    
    #cv2.imwrite(txt_path,mask)

    img = pil_loader(img_path)
    skg = pil_loader(skg_path)
    seg = pil_loader(seg_path)
    txt = pil_loader(txt_path)
    eroded_seg = pil_loader(eroded_seg_path)
    img, skg, seg, eroded_seg, txt = transform([img, skg, seg, eroded_seg, txt])
    img = img.unsqueeze(0)
    skg = skg.unsqueeze(0)
    txt = txt.unsqueeze(0)
    seg = seg.unsqueeze(0)
    eroded_seg = eroded_seg.unsqueeze(0)
    data = [img, skg, seg, eroded_seg, txt]
    
    return data

In [8]:
# skg_path = sketch_image
# img = cv2.imread(skg_path)
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# thresh = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
# edges = cv2.dilate(cv2.Canny(thresh,0,255), None)
# cnt = sorted(cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2], key=cv2.contourArea)[-1]


In [9]:
def load_network(model, save_path):
        
    model_state = torch.load(save_path)
    
    if "state_dict" in model_state:
        model.load_state_dict(model_state["state_dict"])
    else:
        model.load_state_dict(model_state)

        model_state = {
            'state_dict': model.cpu().state_dict(),
            'epoch': epoch,
            'iteration': iteration,
            'model': args.model,
            'color_space': args.color_space,
            'batch_size': args.batch_size,
            'dataset': dataset,
            'image_size': args.image_size
        }
    
    model.cpu()


In [10]:
def get_input(val_loader,xcenter,ycenter,patch_size,num_patch):
    img, skg, seg, eroded_seg, txt = val_loader
    img = custom_transforms.normalize_lab(img)
    skg = custom_transforms.normalize_lab(skg)
    txt = custom_transforms.normalize_lab(txt)
    seg = custom_transforms.normalize_seg(seg)
    eroded_seg = custom_transforms.normalize_seg(eroded_seg)

    bs, w, h = seg.size()

    seg = seg.view(bs, 1, w, h)
    seg = torch.cat((seg, seg, seg), 1)

    eroded_seg = eroded_seg.view(bs, 1, w, h)
    eroded_seg = torch.cat((eroded_seg, eroded_seg, eroded_seg), 1)

    temp = torch.ones(seg.size()) * (1 - seg).float()
    temp[:, 1, :, :] = 0  # torch.ones(seg[:,1,:,:].size())*(1-seg[:,1,:,:]).float()
    temp[:, 2, :, :] = 0  # torch.ones(seg[:,2,:,:].size())*(1-seg[:,2,:,:]).float()

    txt = txt.float() * seg.float() + temp

    patchsize = args.local_texture_size
    batch_size = bs
    if xcenter < 0 or ycenter < 0:
        inp, texture_loc = gen_input_rand(txt, skg, eroded_seg[:, 0, :, :] * 100,
                                              patch_size, patch_size,
                                              num_patch)
    else:
        inp, texture_loc = gen_input_exact(txt, skg, eroded_seg[:, 0, :, :] * 100,xcenter,ycenter,patch_size,1)
        
    return inp,texture_loc 

def get_inputv(inp):
    input_stack = torch.FloatTensor().cpu()
    input_stack.resize_as_(inp.float()).copy_(inp)
    inputv = Variable(input_stack)
    return inputv

In [11]:
def output(sketch, texture):
    
    color_space = 'lab'

    #data = valLoader.__iter__().__next__()
#     input_img1 = cv2.imread("data/78.jpeg")
#     input_img2 = cv2.imread("data/sea.jpeg")
    
    data = convertToStandard(sketch, texture)
    img, skg, seg, eroded_seg, txt = data

    img = custom_transforms.normalize_lab(img)
    skg = custom_transforms.normalize_lab(skg)
    txt = custom_transforms.normalize_lab(txt)
    seg = custom_transforms.normalize_seg(seg)
    eroded_seg = custom_transforms.normalize_seg(eroded_seg)
    inp,texture_loc = get_input(data,-1,-1,30,1)

    seg = seg!=0
    
    #model_location = 'trained_model/final_cloth_finetune.pth'
    model_location = 'C:/Users/ASUS/Desktop/OurGAN/trained_model/final_cloth_finetune.pth'


    netG = texturegan.TextureGAN(5, 3, 32)
    load_network(netG, model_location)
    netG.eval()
    
    model = netG

    inpv = get_inputv(inp.cpu())
    output = model(inpv.cpu())

    out_img = vis_image(custom_transforms.denormalize_lab(output.data.double().cpu()),
                                        color_space)
    inp_img = vis_patch(custom_transforms.denormalize_lab(txt.cpu()),
                                custom_transforms.denormalize_lab(skg.cpu()),
                                texture_loc,
                                color_space)
    tar_img = vis_image(custom_transforms.denormalize_lab(img.cpu()),
                            color_space)

    #plt.figure()
#     plt.imshow(np.transpose(inp_img[0],(1, 2, 0)))
#     plt.axis('off')
    #plt.figure()  
#     plt.figure()
  #  plt.imshow(np.transpose(out_img[0],(1, 2, 0)))
   # plt.axis('off')
    return np.transpose(out_img[0],(1, 2, 0))

In [12]:
#output(sketch, texture)

In [ ]:
sketch = gr.inputs.Image()
texture = gr.inputs.Image()

iface = gr.Interface(fn=output, inputs=[sketch, texture], outputs="image")
iface.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://38851.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "C:\Users\ASUS\anaconda3\lib\site-packages\gradio\networking.py", line 237, in predict
    prediction, durations = app.interface.process(raw_input)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\gradio\interface.py", line 411, in process
    predictions, durations = self.run_prediction(
  File "C:\Users\ASUS\anaconda3\lib\site-packages\gradio\interface.py", line 374, in run_prediction
    prediction = predict_fn(*processed_input)
  File "C:\Users\ASUS\AppData\Local\Temp/ipykernel_4224/3873811009.py", line 9, in output
    data = convertToStandard(sketch, texture)
  File "C:\Users\ASUS\AppData\Local\Temp/ipykernel_4224/3769621804.py", line 14, in convertToStandard
    plt.imshow(img_path)
  File "C:\Users\ASUS\anaconda3\lib\site-packages\matplotlib\pyplot.py", line 2903, in imshow
    __ret = gca().imshow(
  File "C:\Users\ASUS\anaconda3\lib\site-packages\matplotlib\__init__.py", line 1361, in inner
    return func(ax, *map(sanitize_sequence,